# Tutorial: Training

### How to train a model with PyTorch Lightning

* Define your Model Class (in `classification/models`)
    * just as usual, as a subclass of `nn.Module`, where you define architecture in the initializer and implement the forward path in `forward(self, x)`
    * the outputs of the forward path must be of shape `[BATCH_SIZE, NUM_CLASSES]` 
* Also define your subclass of `PLModule`, i.e., PyTorch Lightning Module
    * the PyTorch Lightning Module combines your model with the DataLoader and Solver specifics
    * PyTorch Lightning will then run the training loop, log to TensorBoard, save checkpoints, etc.
    * to create your PyTorch Lightning Module, you can simply inherit from `GeneralPLModule` and set `self.model` to your model
    * if you want to customize further, you can overwrite functions from `GeneralPLModule`:
       * e.g. `prepare_data(self)`, which sets `self.dataset["train"]` and `self.dataset["val"]`

## Adversarial Training

If you want to perform adversarial training, set `model.attack = True`.

You can also pass a custom attack (set `model.attack_function`) and arguments for that attack (set `model.attack_args`).

The attack function must take a whole batch of inputs (if it only take a single input, simply change batchsize to 1) and return the perturbed inputs.

In [ ]:
from attacks.FGA_Batch import fast_gradient_attack 

model_adv = M5PLModule(hparams)
model_adv.attack_fn = fast_gradient_attack
model_adv.attack_args = {"norm":"inf", "epsilon":[0.01, 0.2]}

trainer = pl.Trainer(
    max_epochs=2,
    logger= loggers.TensorBoardLogger(config.LOG_DIR, name="M5"),
    gpus=1 if torch.cuda.is_available() else None,
    log_gpu_memory='all'
)
trainer.fit(model_adv)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)


Loading cached train data from /nfs/students/summer-term-2020/project-4/data/data_8k
Loading cached val data from /nfs/students/summer-term-2020/project-4/data/data_8k


Set SLURM handle signals.

   | Name           | Type         | Params
--------------------------------------------
0  | model          | M5           | 555 K 
1  | model.model    | Sequential   | 555 K 
2  | model.model.0  | Conv1d       | 10 K  
3  | model.model.1  | BatchNorm1d  | 256   
4  | model.model.2  | MaxPool1d    | 0     
5  | model.model.3  | Dropout      | 0     
6  | model.model.4  | Conv1d       | 49 K  
7  | model.model.5  | BatchNorm1d  | 256   
8  | model.model.6  | MaxPool1d    | 0     
9  | model.model.7  | Dropout      | 0     
10 | model.model.8  | Conv1d       | 98 K  
11 | model.model.9  | BatchNorm1d  | 512   
12 | model.model.10 | MaxPool1d    | 0     
13 | model.model.11 | Dropout      | 0     
14 | model.model.12 | Conv1d       | 393 K 
15 | model.model.13 | BatchNorm1d  | 1 K   
16 | model.model.14 | MaxPool1d    | 0     
17 | model.model.15 | AvgPool1d    | 0     
18 | model.model.16 | PermuteLayer | 0     
19 | model.model.17 | Linear       | 1 K   
/nfs

/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Val-Acc=0.03793716656787196


Val-Acc=0.7830468286899822


In [ ]:
model.report()
print("\nAttack:")
_ = model.report(attack=fast_gradient_attack, attack_args = {"norm":"inf", "epsilon":0.1})

In [ ]:
from attacks.FGA_Batch import fast_gradient_attack 
from utils.Visual import drawPlot

# load model. 
#loaded_dict = torch.load("/nfs/homedirs/herrmanp/project-4/experiments/notebooks/pascal/first_search_4.pt")
#model = M5PLModule(loaded_dict["hparams"])
#model.model.load_state_dict(loaded_dict["state_dict"])

# define search space
norms = ["inf"]
epsilons = [1e-100, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1]

# run experiment
all_attack_reports = []
for norm in norms:
    current_attack_report = []
    for eps in epsilons:
        result = model.report(attack=fast_gradient_attack, attack_args = {"norm":norm, "epsilon":eps})
        current_attack_report.append(result)
    all_attack_reports.append(current_attack_report)

    # visalize results
vis_objects = []
for i in range(len(norms)):
    report_accs = [ res['acc'] for res in all_attack_reports[i] ] 
    vis_objects.append({"data": report_accs, "color" : "rbgycmk"[i], "label": "Projected Gradient Descent ({} Norm) after Adv Training (300 epochs, L1, eps=[2000, 20000])".format(norms[i])})
    
drawPlot(x = epsilons, data = vis_objects, x_label = "", y_label = "", title = "")

### Save Model

In [ ]:
'''
from utils.Visual import showAudio
from torch.utils.data import DataLoader  
loaded_dict = torch.load("/nfs/homedirs/herrmanp/project-4/experiments/notebooks/pascal/adv_totaly_hig_50epochs.pt")
best_model = M5(loaded_dict["hparams"])
best_model.load_state_dict(loaded_dict["state_dict"])
best_model.cuda()

test_loader = DataLoader(model.dataset["val"], batch_size=1, shuffle=True)

showAudio(x)
perturbed = model.attack_fn(best_model, x.cuda(), y.cuda(), **{"norm":"inf", "epsilon":0.5})
print(perturbed)
showAudio(perturbed.cpu().numpy())
'''